<a href="https://colab.research.google.com/github/qn19325/individualProject/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import csv
import cv2
import numpy as np

In [ ]:
images = []
speeds = []
with open('data.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        imagepath = 'trainingData/' + str(row[0])
        image = cv2.imread(imagepath, cv2.IMREAD_GRAYSCALE)
        images.append(image)
        speeds.append(row[1])

In [ ]:
images = np.array(images, dtype="float") / 255.0
images = torch.from_numpy(images)

speeds = np.array(speeds, dtype="float32")
speeds = torch.from_numpy(speeds)

print(images.shape)
print(speeds.shape)

torch.Size([252, 64, 64])
torch.Size([252])


In [ ]:
batch_size = 10
n_iters = 252
num_epochs = n_iters / (len(speeds) / batch_size)
num_epochs = int(num_epochs)

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim

        self.layer_dim = layer_dim

        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        out, hn = self.rnn(x, h0.detach())
 
        out = self.fc(out[:, -1, :]) 
        
        return out

In [ ]:
input_dim = 4096
hidden_dim = 100
layer_dim = 1
output_dim = 10

In [ ]:
model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [ ]:
# Number of steps to unroll
seq_dim = 252

for epoch in range(num_epochs):
    for i in range(len(speeds)):
        model.train()
        # Load images as tensors with gradient accumulation abilities
        image = images[i]
        image = images.view(-1, seq_dim, input_dim).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images.float())
        print(outputs)
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, speeds.long())

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

RuntimeError: ignored